## 그룹연산

- 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나눠서 관찰하는 것도 좋은 방법이다.
- 특정 기준을 적용하여 몇개의 그룹으로 분할하여 처리하는 것을 그룹연산이라고 부른다.
- 특히 그룹연산은 데이터를 집계, 변환, 필터링하는데 효율적이다. 데이터 그룹을 분할하는 1단계 과정은 판다스 groupby() 메소드를 사용한다.
1. 분할(split): 데이터를 특정 조건에 의해 분할
2. 적용(apply): 데이터를 집계, 변환, 필터링하는데 필요한 메소드를 적용
3. 결합(combine): 2단계의 처리 결과를 하나로 결합

### 1. 그룹 객체 만들기(분할 단계)

1개의 열을 기준으로 그룹화 : DataFrame.groupby(기준이되는열)

titanic 데이터셋에서 class열을 기준으로 하여 그룹으로 나눈다.

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]

print('승객수:', len(df))

승객수: 891


In [3]:
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [7]:
grouped = df.groupby(['class'])

df_cls 그룹객체에서 반복문을 통해 first, second, third 데이터 값의 개수를 알수있다.

In [9]:
for key, group in grouped:
    print('key:', key)
    print('number:', len(group))
    print(group.head())
    print('\n')

key: First
number: 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


key: Second
number: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


key: Third
number: 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




앞서 생성한 그룹 객체인 grouped에 연산 메소드를 적용할 수 있다. 그룹별 평균값을 구하기 위해 mean() 메소드를 적용하자.

In [12]:
average = grouped.mean()
print(average) # 범주형 컬럼은 제외

              age       fare  survived
class                                 
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363
